In [247]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

import calliope

# We increase logging verbosity
calliope.set_log_verbosity("INFO", include_solver_output=False)

model = calliope.read_yaml("model.yaml")

[2025-10-13 10:05:20] INFO     Math init | loading pre-defined math.


[2025-10-13 10:05:20] INFO     Math init | loading math files {'spores', 'storage_inter_cluster', 'operate', 'milp', 'base'}.
[2025-10-13 10:05:20] INFO     Model: preprocessing data
[2025-10-13 10:05:20] INFO     Math build | building applied math with ['base'].
[2025-10-13 10:05:21] INFO     input data `color` not defined in model math; it will not be available in the optimisation problem.
[2025-10-13 10:05:21] INFO     input data `name` not defined in model math; it will not be available in the optimisation problem.
[2025-10-13 10:05:21] WARNING  ModelWarning: Only one timestep defined. Inferring timestep resolution to be 1 hour

[2025-10-13 10:05:21] INFO     input data `color` not defined in model math; it will not be available in the optimisation problem.
[2025-10-13 10:05:21] INFO     input data `name` not defined in model math; it will not be available in the optimisation problem.
[2025-10-13 10:05:21] INFO     Model: initialisation complete


In [248]:
model.inputs

<xarray.Dataset> Size: 3kB
Dimensions:                     (costs: 1, techs: 10, nodes: 6, carriers: 2,
                                 timesteps: 1)
Coordinates:
  * costs                       (costs) object 8B 'monetary'
  * techs                       (techs) object 80B 'E1_to_D1' ... 'supply_gri...
  * carriers                    (carriers) object 16B 'electricity' 'heat'
  * nodes                       (nodes) object 48B 'D1' 'D2' 'E1' 'H1' 'S1' 'S2'
  * timesteps                   (timesteps) datetime64[ns] 8B 2050-01-01
Data variables: (12/23)
    cost_interest_rate          (costs) float64 8B 0.1
    bigM                        float64 8B 1e+06
    objective_cost_weights      (costs) float64 8B 1.0
    base_tech                   (techs) object 80B 'transmission' ... 'supply'
    carrier_in                  (nodes, techs, carriers) bool 120B True ... F...
    color                       (techs) object 80B '#6783E3' ... '#C5ABE3'
    ...                          ...
    longitude                   (nodes) float64 48B 4.361 4.362 ... 4.356 4.357
    sink_use_equals             (timesteps, techs, nodes) float64 480B nan .....
    definition_matrix           (nodes, techs, carriers) bool 120B True ... F...
    distance                    (techs) float64 80B 0.1734 0.2448 ... nan nan
    timestep_resolution         (timesteps) float64 8B 1.0
    timestep_weights            (timesteps) float64 8B 1.0

In [249]:
model.inputs.flow_cap_max.to_series().dropna()

techs              carriers   
E1_to_D1           electricity    2000.0
E1_to_D2           electricity    2000.0
H1_to_D1           heat           2000.0
H1_to_D2           heat           2000.0
S1_to_H1           heat           2000.0
S2_to_E1           electricity    2000.0
heatpump           heat             50.0
supply_geothermal  heat           2000.0
supply_grid_power  electricity    2000.0
Name: flow_cap_max, dtype: float64

In [250]:
model.inputs.sink_use_equals.sum(
    "timesteps", min_count=1, skipna=True
).to_series().dropna()

techs        nodes
demand_heat  D1       150.0
             D2       200.0
Name: sink_use_equals, dtype: float64

In [251]:
model.build()
model.solve()

[2025-10-13 10:05:22] INFO     Model: backend build starting
[2025-10-13 10:05:22] INFO     Optimisation Model | parameters/lookups | Generated.
[2025-10-13 10:05:23] INFO     Optimisation Model | variables | Generated.
[2025-10-13 10:05:25] INFO     Optimisation Model | global_expressions | Generated.
[2025-10-13 10:05:27] INFO     Optimisation Model | constraints | Generated.
[2025-10-13 10:05:27] INFO     Optimisation Model | piecewise_constraints | Generated.
[2025-10-13 10:05:27] INFO     Optimisation Model | objectives | Generated.
[2025-10-13 10:05:27] INFO     Model: backend build complete
[2025-10-13 10:05:27] INFO     Optimisation model | starting model in base mode.
[2025-10-13 10:05:28] INFO     Backend: solver finished running. Time since start of solving optimisation problem: 0:00:00.869445
[2025-10-13 10:05:28] INFO     Postprocessing: applied zero threshold 1e-10 to model results.
[2025-10-13 10:05:28] INFO     Postprocessing: ended. Time since start of solving optimisa

In [252]:
model.results

<xarray.Dataset> Size: 10kB
Dimensions:                     (nodes: 6, techs: 10, carriers: 2,
                                 timesteps: 1, costs: 1)
Coordinates:
  * techs                       (techs) object 80B 'E1_to_D1' ... 'supply_gri...
  * carriers                    (carriers) object 16B 'electricity' 'heat'
  * nodes                       (nodes) object 48B 'D1' 'D2' 'E1' 'H1' 'S1' 'S2'
  * timesteps                   (timesteps) datetime64[ns] 8B 2050-01-01
  * costs                       (costs) object 8B 'monetary'
Data variables: (12/21)
    flow_cap                    (nodes, techs, carriers) float64 960B 53.71 ....
    link_flow_cap               (techs) float64 80B 53.71 53.71 ... nan nan
    flow_out                    (nodes, techs, carriers, timesteps) float64 960B ...
    flow_in                     (nodes, techs, carriers, timesteps) float64 960B ...
    source_use                  (nodes, techs, timesteps) float64 480B nan .....
    source_cap                  (nodes, techs) float64 480B nan nan ... 109.6
    ...                          ...
    min_cost_optimisation       float64 8B 0.4912
    capacity_factor             (nodes, techs, carriers, timesteps) float64 960B ...
    systemwide_capacity_factor  (techs, carriers) float64 160B 0.49 0.0 ... 0.0
    systemwide_levelised_cost   (techs, costs, carriers) float64 160B 2.226e-...
    total_levelised_cost        (costs, carriers) float64 16B 0.004481 0.001392
    unmet_sum                   (nodes, carriers, timesteps) float64 96B 0.0 ...

In [253]:
df_heat = (
    model.results.flow_out.sel(carriers="heat")
    .sum("nodes", min_count=1, skipna=True)
    .to_series()
    .dropna()
    .unstack("techs")
)

df_heat.head()

techs,H1_to_D1,H1_to_D2,S1_to_H1,heatpump,supply_geothermal
timesteps,,,,,
2050-01-01,100.0,150.0,251.372489,100.0,252.828685


In [254]:
costs = model.results.cost.to_series().dropna()
costs.head()

nodes  techs     costs   
D1     E1_to_D1  monetary    5.857036e-07
       H1_to_D1  monetary    3.286138e-05
       heatpump  monetary    0.000000e+00
D2     E1_to_D2  monetary    8.265942e-07
       H1_to_D2  monetary    6.969086e-05
Name: cost, dtype: float64

In [255]:
# We set the color mapping to use in all our plots by extracting the colors defined in the technology definitions of our model.
colors = model.inputs.color.to_series().to_dict()

df_electricity = (
    (model.results.flow_out.fillna(0) - model.results.flow_in.fillna(0))
    .sel(carriers="electricity")
    .sum("nodes")
    .to_series()
    .where(lambda x: x != 0)
    .dropna()
    .to_frame("Flow in/out (kWh)")
    .reset_index()
)
df_electricity_demand = df_electricity[df_electricity.techs == "demand_electricity"]
df_electricity_other = df_electricity[df_electricity.techs != "demand_electricity"]

print(df_electricity.head())

fig1 = px.bar(
    df_electricity_other,
    x="timesteps",
    y="Flow in/out (kWh)",
    color="techs",
    color_discrete_map=colors,
)
fig1.add_scatter(
    x=df_electricity_demand.timesteps,
    y=-1 * df_electricity_demand["Flow in/out (kWh)"],
    marker_color="black",
    name="demand",
)

               techs  timesteps  Flow in/out (kWh)
0           E1_to_D1 2050-01-01          -1.074114
1           E1_to_D2 2050-01-01          -1.074114
2           S2_to_E1 2050-01-01          -2.192069
3           heatpump 2050-01-01        -105.263158
4  supply_grid_power 2050-01-01         109.603455


In [256]:
carriers = ["heat", "electricity"]
df_flows = (
    (model.results.flow_out.fillna(0) - model.results.flow_in.fillna(0))
    .sel(carriers=carriers)
    .to_series()
    .where(lambda x: x != 0)
    .dropna()
    .to_frame("Flow in/out (kWh)")
    .reset_index()
)
df_demand = df_flows[df_flows.techs.str.contains("demand")]
df_flows_other = df_flows[~df_flows.techs.str.contains("demand")]

print(df_flows.head())

node_order = df_flows_other.nodes.unique()

fig = px.bar(
    df_flows_other,
    x="timesteps",
    y="Flow in/out (kWh)",
    facet_row="nodes",
    facet_col="carriers",
    color="techs",
    category_orders={"nodes": node_order, "carriers": carriers},
    height=1000,
    color_discrete_map=colors,
)

showlegend = True
# we reverse the node order (`[::-1]`) because the rows are numbered from bottom to top.
for row, node in enumerate(node_order[::-1]):
    for col, carrier in enumerate(carriers):
        demand_ = df_demand.loc[
            (df_demand.nodes == node) & (df_demand.techs == f"demand_{carrier}"),
            "Flow in/out (kWh)",
        ]
        if not demand_.empty:
            fig.add_scatter(
                x=model.results.timesteps.values,
                y=-1 * demand_,
                row=row + 1,
                col=col + 1,
                marker_color="black",
                name="Demand",
                legendgroup="demand",
                showlegend=showlegend,
            )
            showlegend = False
fig.update_yaxes(matches=None)
fig.show()

  nodes        techs     carriers  timesteps  Flow in/out (kWh)
0    D1     E1_to_D1  electricity 2050-01-01          52.631579
1    D1     H1_to_D1         heat 2050-01-01         100.000000
2    D1  demand_heat         heat 2050-01-01        -150.000000
3    D1     heatpump         heat 2050-01-01          50.000000
4    D1     heatpump  electricity 2050-01-01         -52.631579


In [257]:
df_capacity = (
    model.results.flow_cap.where(
        ~model.inputs.base_tech.str.contains("demand|transmission")
    )
    .to_series()
    .where(lambda x: x != 0)
    .dropna()
    .to_frame("Flow capacity (kW)")
    .reset_index()
)

print(df_capacity.head())

fig = px.bar(
    df_capacity,
    x="nodes",
    y="Flow capacity (kW)",
    color="techs",
    facet_col="carriers",
    color_discrete_map=colors,
)
fig.show()

  nodes              techs     carriers  Flow capacity (kW)
0    D1           heatpump  electricity           52.631579
1    D1           heatpump         heat           50.000000
2    D2           heatpump  electricity           52.631579
3    D2           heatpump         heat           50.000000
4    S1  supply_geothermal         heat          252.828685


In [265]:
df_coords = model.inputs[["latitude", "longitude"]].to_dataframe().reset_index()
df_capacity = (
    model.results.flow_cap.where(model.inputs.base_tech == "transmission")
    .to_series()
    .where(lambda x: x != 0)
    .dropna()
    .to_frame("Flow capacity (kW)")
    .reset_index()
)
df_capacity_coords = pd.merge(df_coords, df_capacity, left_on="nodes", right_on="nodes").sort_values(by=['techs'])
fig1 = px.line_map(
    df_capacity_coords,
    lat="latitude",
    lon="longitude",
    color="carriers",
    hover_name="nodes",
    hover_data="Flow capacity (kW)",
    zoom=3,
    height=2000,
)
fig2 = px.scatter_map(
    df_capacity_coords,
    lat="latitude",
    lon="longitude",
    color="carriers",
    hover_name="nodes",
    hover_data="Flow capacity (kW)",
    zoom=3,
    height=2000,
)
fig=go.Figure(data = fig1.data + fig2.data)
fig.update_layout(
    map_style="open-street-map",
    map_zoom=16,
    map_center_lat=df_coords.latitude.mean(),
    map_center_lon=df_coords.longitude.mean(),
    margin={"r": 0, "t": 0, "l": 0, "b": 0},
    hoverdistance=50,
)